# Joular and CK data analysis

In [60]:
import requests
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from scipy import stats

BASE_URL = "http://localhost:8080/api/v1"

# Exploration of data

- **First objective**: aggregate all the joular data for each method and keep the methods having 30 values
- **Second objective**: remove all the aberrant values like the values that are too high or too low compared to the other values
- **Third objective**: sort all the data in function of their values in descending order
- **Fourth objective**: for a few methods (those having the highest values), compare the distribution of the values with the CK analysis of this method

## Aggregate the joular values and getting the methods with 30 values

In [41]:
def getEndpoint(endpoint, params=None):
    URL = BASE_URL + endpoint
    r = requests.get(url=URL, params=params)
    return r.json()

def getAllJoularDataForMethodsHaving30Values(sha=""):
    allResults = []
    endpoint = "/joular/aggregates/"
    if (sha != ""):
        endpoint += "by-commit/" + sha
    result = getEndpoint(endpoint)
    allResults.extend(result["content"])
    while(not result["last"]):
        page = result["number"] + 1
        print("Calling page ", page)
        result = getEndpoint(endpoint, {"page": page})
        allResults.extend(result["content"])
    return allResults

def getSeveralCkDataForOneMethod(commitSha: str, className: str, methodSignature: str, ckMetrics: list[str]):
    endpoint = "/ck-entities/by-commit-and-ast-elem/" + commitSha
    params = {
        "className": className,
        "methodSignature": methodSignature,
        "names": ckMetrics
    }
    return getEndpoint(endpoint=endpoint, params=params)

def createBoxplotJoular(repositoryName, methodSignature, allValues):
    df = pd.DataFrame({"allValues":allValues})
    fig = px.box(df, y="allValues", title="Joular values for " + methodSignature + " of repository " +repositoryName, points="outliers")
    fig.show()

In [3]:
def removeOutliersByZScore(data, threshold=3):
    zScores = np.abs(stats.zscore(data))
    #zScores = np.abs((data - np.mean(data)) / np.std(data))
    """boolScore = zScores < threshold
    for i in range(len(data)):
        print(str(data[i]) + "   " + str(zScores[i]) + "  " + str(boolScore[i]))"""
    return data[zScores < threshold]

In [4]:
def removeOutliersByIQR(allValues):
    df = pd.DataFrame({"allValues":allValues})
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    """print("Q1 = ", Q1["allValues"])
    print("Q3 = ", Q3["allValues"])
    print("IQR = ", IQR["allValues"])
    print("Lower limit = ", Q1["allValues"] - 1.5 * IQR["allValues"])
    print("Upper limit = ", Q3["allValues"] + 1.5 * IQR["allValues"])"""
    return df[~((df < (Q1 - 1.5 * IQR))|(df > (Q3 + 1.5 * IQR))).any(axis=1)]

### Data for all repositories

In [9]:
allData = getAllJoularDataForMethodsHaving30Values()
print(allData)
dataRepo = sorted(allData, key=lambda x: x["allValues"], reverse=True)
dataRepo = []
for methodData in dataRepo:
    repositoryName = methodData["commit"]["repository"]["name"]
    methodSignature = methodData["measurableElement"]["methodSignature"]
    allValues = methodData["allValues"]
    allValuesAfterIQR = removeOutliersByIQR(allValues)
    #print(len(allValuesAfterIQR))
    if (len(allValuesAfterIQR) == 30):
        dataRepo.append(methodData)

    #createBoxplotJoular(repositoryName, methodSignature, allValues)
    #createBoxplotJoular(repositoryName, methodSignature, allValuesWithoutOutliers)
    #createBoxplotJoular(repositoryName, methodSignature, allValuesAfterIQR["allValues"])
print(len(dataRepo))

dataAllRepositories = []
for doc in dataRepo:
    dataAllRepositories.extend({"Repository": doc["commit"]["repository"]["name"], "Values":val, "Class": doc["measurableElement"]["className"]} for val in doc["allValues"])
ckDataFrame = pd.DataFrame(dataAllRepositories)

figRepository = px.violin(ckDataFrame, y="Values", x="Repository", points="all")
figRepository.show()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Aggregate by repository and comparison with CK values

In [110]:
def sortData(data):
    return sorted(data, key=lambda x: x["allValues"], reverse=True)

def removeOutliers(data):
    print("Len with outliers : ", len(data))
    only30Values = []
    for methodData in data:
        allValues = methodData["allValues"]
        allValuesAfterIQR = removeOutliersByIQR(allValues)
        #print(len(allValuesAfterIQR))
        if (len(allValuesAfterIQR) == 30):
            only30Values.append(methodData)
    print("Len without outliers : ", len(only30Values))
    return only30Values

def getTheFirstHighestValues(data, end=5):
    return data[:end]

def createViolinJoular(data, granularity="Repository", points="all"):
    dataAllRepositories = []
    for doc in data:
        dataAllRepositories.extend({"Repository": doc["commit"]["repository"]["name"], "Values":val, "Class": doc["measurableElement"]["className"]} for val in doc["allValues"])
    df = pd.DataFrame(dataAllRepositories)

    figRepository = px.violin(df, y="Values", x=granularity, points=points)
    figRepository.show()

def createMultipleBoxplot(dataOneRepo):
    dataframePreparation = []
    for method in dataOneRepo:
        dataframePreparation.append({"Method": method["measurableElement"]["methodSignature"], "Values":method["allValues"], "Class":method["measurableElement"]["className"]})
    df = pd.DataFrame(dataframePreparation)
    print(df)
    fig = px.box(df, x="Method", y="Values", points="outliers")
    fig.show()
        


def createBarChart(dataframe, xAxisName, yAxisName, title):
    fig = px.bar(dataframe, x=xAxisName, y=yAxisName, title=title)
    fig.update_layout(yaxis_range=[0, 40])
    return fig

def prepareCkDataFrame(ckValues):
    values = {item["name"]:item["value"] for item in ckValues}
    return pd.DataFrame(list(values.items()), columns=["Metric", "Value"])

def createBoxplot(dataframe, title):
    return px.box(dataframe, y="value", title=title, points="all")

def prepareJoularDataFrame(joularValues):
    return pd.DataFrame({"value": joularValues})

def createSubplot(fig1, fig2, title):
    fig1Traces = []
    fig2Traces = []
    for trace in range(len(fig1["data"])):
        fig1Traces.append(fig1["data"][trace])
    for trace in range(len(fig2["data"])):
        fig2Traces.append(fig2["data"][trace])

    
    figure = make_subplots(rows=1, cols=2, subplot_titles=("Joular values", "Ck values"))
    for traces in fig1Traces:
        figure.append_trace(traces, row=1, col=1)
    for traces in fig2Traces:
        figure.append_trace(traces, row=1, col=2)

    figure.update_layout(title_text=title)
    figure.show()

In [111]:
allCommitSha = ["066f4cf207359e06d30911a553dedd054aef595c", "5c9d8989f968d0ee3a942b411ef7fe121ed94609", "12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "3ed1f1a064a10e53adc2ad8c0b46a4b2c148ee21", "59e5152722198526c6ffe5361de7d1a6a87275c7"]
allowedCkMetrics = ["cbo", "loc", "wmc"]

for sha in allCommitSha:
    dataRepo = getAllJoularDataForMethodsHaving30Values(sha)
    sortedDataRepo = sortData(dataRepo)
    dataRepoWithoutOutliers = removeOutliers(sortedDataRepo)
    #createViolinJoular(dataRepoWithoutOutliers)
    dataRepoWithoutOutliers = getTheFirstHighestValues(dataRepoWithoutOutliers, 5)
    createMultipleBoxplot(dataRepoWithoutOutliers)
    #createViolinJoular(dataRepoWithoutOutliers, "Class", "outliers")

    """for i in range(5):
        fir = make_subplots(rows=1, cols=2)
        # JOULAR DATA
        joularData = dataRepoWithoutOutliers[i]["allValues"]
        joularDataFrame = prepareJoularDataFrame(joularData)
        figBoxPlot = createBoxplot(joularDataFrame, "Joular values for " + methodSignature + " (" + className + ")")

        # CK DATA
        className = dataRepoWithoutOutliers[i]["measurableElement"]["className"]
        methodSignature = dataRepoWithoutOutliers[i]["measurableElement"]["methodSignature"]
        ckValues = getSeveralCkDataForOneMethod(commitSha=sha, className=className, methodSignature=methodSignature, ckMetrics=allowedCkMetrics)
        ckDataFrame = prepareCkDataFrame(ckValues)
        figBarChart = createBarChart(ckDataFrame, "Metric", "Value", "Ck metrics for " + methodSignature + " (" + className + ")")

        # SUB PLOT
        title = "Method " + methodSignature + " of the class " + className
        createSubplot(figBoxPlot, figBarChart, title)"""


Calling page  1
Len with outliers :  197
Len without outliers :  93
                                              Method  \
0  scan/2[spoon.reflect.path.CtRole,java.util.Col...   
1  scanCtReference/1[spoon.support.visitor.equals...   
2  save/2[spoon.support.Factory,java.io.OutputStr...   
3                                     getModifiers/0   
4                  setSimpleName/1[java.lang.String]   

                                              Values  \
0  [2109.9353, 2580.8672, 1992.2736, 2085.4592, 2...   
1  [324.4667, 319.6152, 261.6414, 287.9847, 364.9...   
2  [181.0151, 248.0991, 178.4615, 172.9383, 202.6...   
3  [137.1716, 174.9281, 145.9525, 148.2249, 131.9...   
4  [125.4578, 187.7617, 145.0521, 110.2401, 158.6...   

                                             Class  
0    spoon.reflect.visitor.EarlyTerminatingScanner  
1       spoon.support.visitor.equals.EqualsChecker  
2         spoon.support.SerializationModelStreamer  
3          spoon.support.reflect.CtModifierHan

Len with outliers :  76
Len without outliers :  32
                                              Method  \
0  correlateByWords/2[java.lang.String,java.lang....   
1                                            setUp/0   
2  writeToIndex/2[org.jabref.logic.pdf.search.ind...   
3                                   receiveMessage/0   
4  createMarshallerAndWriteToFile/2[org.jabref.lo...   

                                              Values  \
0  [109.749405, 123.503296, 105.668396, 122.21850...   
1  [108.6305, 98.3913, 98.205, 97.3779, 94.3094, ...   
2  [85.19861, 196.84381, 72.4242, 182.2456, 248.1...   
3  [84.8567, 45.9535, 40.7698, 65.6244, 93.5691, ...   
4  [70.2172, 61.227303, 58.006897, 72.5075, 85.41...   

                                             Class  
0         org.jabref.logic.database.DuplicateCheck  
1                org.jabref.cli.AuxCommandLineTest  
2  org.jabref.logic.pdf.search.indexing.PdfIndexer  
3          org.jabref.logic.remote.shared.Protocol  
4       or

Len with outliers :  73
Len without outliers :  15
                                      Method  \
0              findClass/1[java.lang.String]   
1                                     init/0   
2  resolveLocalXsdSchema/1[java.lang.String]   
3              findClass/1[java.lang.String]   
4         testSupportsRefCursorsAboveJava8/0   

                                              Values  \
0  [463.8108, 508.5263, 466.7141, 608.1737, 509.7...   
1  [79.2474, 33.324303, 99.7014, 20.7368, 97.6481...   
2  [45.2483, 34.6892, 38.3209, 43.2398, 29.461401...   
3  [40.5933, 15.9507, 48.4024, 6.5783, 31.0427, 1...   
4  [38.6331, 28.5637, 31.8307, 15.5525, 52.7319, ...   

                                               Class  
0  org.hibernate.orm.test.bootstrap.registry.clas...  
1  org.hibernate.orm.test.internal.util.ReflectHe...  
2            org.hibernate.boot.xsd.LocalXsdResolver  
3  org.hibernate.boot.registry.classloading.inter...  
4  org.hibernate.orm.test.jdbc.refcursor.Standar

# Test 

In [2]:
def getJoularDataFromOneMethodFromOneCommit(commitSha, className, methodSignature):
    URL = BASE_URL + "/joular-entities/by-commit-and-ast-elem/" + commitSha
    PARAMS = {"className":className, "methodSignature":methodSignature}
    r = requests.get(url = URL, params = PARAMS)
    return r.json()

In [3]:
def getCkDataFromOneCommit(commitSha, astElem, className, methodSignature=None):
    URL = BASE_URL + "/ck-entities/by-commit-and-ast-elem/" + commitSha
    if methodSignature != None:
        PARAMS = {"astElem": astElem, "className":className, "methodSignature":methodSignature}
    else:
        PARAMS = {"astElem": astElem, "className":className}
    r = requests.get(url = URL, params = PARAMS)
    return r.json()

In [4]:
def createJoularBoxplot(commitSha, className, methodSignature):
    data = getJoularDataFromOneMethodFromOneCommit(commitSha, className, methodSignature)
    values = [item["value"] for item in data["content"] if item["value"] > 0]
    df = pd.DataFrame({"value":values})
    fig = px.box(df, y="value", title="Joular values for " + methodSignature, points='all')
    fig.show()

In [5]:
def createCkBarChart(commitSha, astElem, className, methodSignature):
    NAMES = ["wmc", "cbo", "cboModified", "loc", "fanin", "fanout"]
    data = getCkDataFromOneCommit(commitSha, astElem, className, methodSignature)
    values = {item["name"]:item["value"] for item in data["content"] if item["name"] in NAMES}
    df = pd.DataFrame(list(values.items()), columns=["Metric", "Value"])
    fig = px.bar(df, x="Metric", y="Value", title="Ck metrics for " + methodSignature)
    fig.show()

In [6]:
def createBoxplotAndBarChart(commitSha, astElem, className, methodSignature):
    createJoularBoxplot(commitSha, className, methodSignature)
    createCkBarChart(commitSha, astElem, className, methodSignature)

In [ ]:
createBoxplotAndBarChart("3ed1f1a064a10e53adc2ad8c0b46a4b2c148ee21", "method", "org.springframework.boot.web.servlet.server.StaticResourceJars", "isResourcesJar/1[java.util.jar.JarFile]")

createBoxplotAndBarChart("12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "method", "org.hibernate.engine.jdbc.internal.ResultSetReturnImpl", "executeUpdate/2[java.sql.PreparedStatement,java.lang.String]")

createBoxplotAndBarChart("12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "method", "org.hibernate.bytecode.internal.bytebuddy.ByteBuddyState", "make/2[TypePool,DynamicType.Builder<?>]")

createBoxplotAndBarChart("12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "method", "org.hibernate.boot.registry.classloading.internal.AggregatedServiceLoader$ClassPathAndModulePathAggregatedServiceLoader", "hasNextIgnoringServiceConfigurationError/1[java.util.Iterator<?>]")

createBoxplotAndBarChart("3ed1f1a064a10e53adc2ad8c0b46a4b2c148ee21", "method", "org.springframework.boot.web.embedded.tomcat.TomcatWebServer", "initialize/0")

createBoxplotAndBarChart("12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "method", "org.hibernate.testing.junit4.FailureExpectedHandler", "evaluate/0")

In [ ]:
createBoxplotAndBarChart("3ed1f1a064a10e53adc2ad8c0b46a4b2c148ee21", "method", "org.springframework.boot.testsupport.classpath.ModifiedClassPathClassLoader", "loadClass/1[java.lang.String]")

In [ ]:
createBoxplotAndBarChart("12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "method", "org.hibernate.boot.registry.classloading.internal.AggregatedClassLoader", "getResources/1[java.lang.String]")

### Data with aberrant negative values

The method *verifyCollection* has the value -530894.

In [12]:
createBoxplotAndBarChart("12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "method", "org.hibernate.orm.test.bootstrap.registry.classloading.PhantomReferenceLeakDetector", "verifyCollection/3[java.lang.ref.ReferenceQueue<T>,int,int]")

The method *accept* below has the value -217371.5

In [13]:
createBoxplotAndBarChart("12442bd8c7cde6e7c006a6277eeb8e81ad0c2219", "method", "org.hibernate.tool.schema.internal.exec.GenerationTargetToDatabase", "accept/1[java.lang.String]")